In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib
import re
import unicodedata

In [2]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [3]:
response  = requests.get(url).text

In [4]:
soup = BeautifulSoup(response, 'html5lib')

In [5]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [6]:
# To extract only tables from wikipedia page
html_tables = soup.find_all('table')

In [7]:
len(html_tables)

24

In [44]:
""" Graphs on wikipedia are also arranged as table 
So table[2]/ 3rd table is our first data table """
first_launch_table = html_tables[2]

In [45]:
# Extracting column name

In [46]:
table_headers = first_launch_table.find_all("th")

In [47]:
table_headers

[<th scope="col">Flight No.
 </th>,
 <th scope="col">Date andtime ()
 </th>,
 <th scope="col"> 
 </th>,
 <th scope="col">Launch site
 </th>,
 <th scope="col">Payload
 </th>,
 <th scope="col">Payload mass
 </th>,
 <th scope="col">Orbit
 </th>,
 <th scope="col">Customer
 </th>,
 <th scope="col">Launchoutcome
 </th>,
 <th scope="col">
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">1
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">2
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">3
 </th>,
 <th rowspan="3" scope="row" style="text-align:center;">4
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">5
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">6
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">7
 </th>]

In [48]:
type(table_headers)

bs4.element.ResultSet

In [87]:
column_names = []
for name in table_headers :
    column_name = ' '.join(name.contents)
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        if column_name is not None and len(column_name) > 0 :
            column_names.append(column_name)        

In [89]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [98]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass



In [90]:
launch_dict= dict.fromkeys(column_names)

In [91]:
del launch_dict['Date and time ( )']

In [94]:
print(launch_dict['Flight No.'])

None


In [124]:
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [125]:
print(launch_dict['Flight No.'])

[]


In [126]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            launch_dict['Flight No.'].append(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload_mass)
            launch_dict['Payload mass'].append(payload_mass)

            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].text.strip()
            #print(customer)
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            launch_dict['Booster landing'].append(booster_landing)
            

In [127]:
for key, value in launch_dict.items() :
    print(f" {key} : {len(value)}")

 Flight No. : 121
 Launch site : 121
 Payload : 121
 Payload mass : 121
 Orbit : 121
 Customer : 121
 Launch outcome : 121
 Version Booster : 121
 Booster landing : 121
 Date : 121
 Time : 121


In [128]:
# del launch_dict['Flight No.'][0]

In [129]:
df=pd.DataFrame(launch_dict)

In [130]:
df.shape

(121, 11)

In [131]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,".mw-parser-output .plainlist ol,.mw-parser-out...",Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA (COTS),Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA (CRS),Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA (CRS),Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10


In [132]:
df.to_csv('SpaceX_wikiweb_scraped.csv', index=False)

### Thank You